SMTP发送邮件
---
SMTP是发送邮件的协议，Python内置对SMTP的支持，可以发送纯文本邮件、HTML邮件以及带附件的邮件。

Python对SMTP支持有smtplib和email两个模块，**email**负责**构造邮件**，**smtplib**负责**发送邮件**。

In [ ]:
# 163->qq测试成功，根据内容注意查看垃圾邮件。
'''构造纯文本邮件'''
from email.mime.text import MIMEText
msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')

'''通过SMTP发出去'''
# 输入Email地址和口令（客户端授权密码）:
from_addr = input('From: ')
password = input('Password: ')
# 输入收件人地址:
to_addr = input('To: ')
# 输入发件邮箱SMTP服务器地址:  smtp.163.com、smtp.qq.com
smtp_server = input('SMTP server: ')

# 防止错误smtplib.SMTPDataError: (554,...： 信封发件人和信头发件人不匹配导致
msg['From'] = from_addr + ' <' + from_addr + '>'

import smtplib
server = smtplib.SMTP(smtp_server, 25) # SMTP协议默认端口是25
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

**MIMEText**第一个参数就是**邮件正文**，第二个参数是MIME的subtype，传入**'plain'表示纯文本**，最终的MIME就是**'text/plain'**，最后一定要用**utf-8编码**保证**多语言兼容性**。

我们用**set_debuglevel(1)**就可以打印出和SMTP服务器**交互的所有信息**。SMTP协议就是简单的文本命令和响应。login()方法用来登录SMTP服务器，sendmail()方法就是发邮件，由于可以一次发给**多个人**，所以传入一个**list**，邮件正文是一个str，as_string()把MIMEText对象变成str。

邮件主题、如何显示发件人、收件人等信息并不是通过SMTP协议发给MTA，而是包含在发给MTA的文本中的，所以，必须把**From**、**To**和**Subject**添加到**MIMEText**中，才是一封完整的邮件。

In [ ]:
# 163->qq测试成功
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr

import smtplib

def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

from_addr = input('From: ')
password = input('Password: ')
to_addr = input('To: ')
smtp_server = input('SMTP server: ')

# msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')
msg = MIMEText('<html><body><h1>Hello</h1>' +
    '<p>send by <a href="http://www.python.org">Python</a>...</p>' +
    '</body></html>', 'html', 'utf-8')
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)
msg['To'] = _format_addr('管理员 <%s>' % to_addr)
msg['Subject'] = Header('来自SMTP的问候……', 'utf-8').encode()

server = smtplib.SMTP(smtp_server, 25)
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

函数_format_addr()来格式化一个邮件地址。注意不能简单地传入name <addr@example.com>，因为如果包含中文，需要通过Header对象进行编码。

msg['To']接收的是**字符串***而不是list，如果有多个邮件地址，用**,分隔**即可。


你看到的收件人的名字很可能不是我们传入的管理员，因为很多邮件服务商在显示邮件时，会把**收件人名字自动替换为用户注册的名字**，但是其他收件人名字的显示不受影响。

如果我们查看Email的原始内容，可以看到如下经过编码的邮件头：

From: =?utf-8?b?UHl0aG9u54ix5aW96ICF?= <xxxxxx@163.com>  
To: =?utf-8?b?566h55CG5ZGY?= <xxxxxx@qq.com>  
Subject: =?utf-8?b?5p2l6IeqU01UUOeahOmXruWAmeKApuKApg==?=  

这就是经过Header对象编码的文本，包含utf-8编码信息和Base64编码的文本。

发送HTML邮件
---
在构造MIMEText对象时，把HTML字符串传进去，再把第二个参数由plain变为**html**就可以了。

In [ ]:
msg = MIMEText('<html><body><h1>Hello</h1>' +
    '<p>send by <a href="http://www.python.org">Python</a>...</p>' +
    '</body></html>', 'html', 'utf-8')

发送附件
---
带附件的邮件可以看做包含若干部分的邮件：**文本**和**各个附件**本身，所以，可以构造一个**MIMEMultipart**对象代表邮件本身，然后往里面加上一个**MIMEText**作为邮件正文，再继续往里面加上表示附件的**MIMEBase**对象即可。

In [ ]:
# 新增import：
from email.mime.multipart import MIMEBase,MIMEMultipart

# 邮件对象:
msg = MIMEMultipart()
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)
msg['To'] = _format_addr('管理员 <%s>' % to_addr)
msg['Subject'] = Header('来自SMTP的问候……', 'utf-8').encode()

# 邮件正文是MIMEText:
#msg.attach(MIMEText('send with file...', 'plain', 'utf-8'))
msg.attach(MIMEText('<html><body><h1>Hello</h1>' +
    '<p><img src="cid:0"></p>' +
    '</body></html>', 'html', 'utf-8'))

# 添加附件就是加上一个MIMEBase，从本地读取一个图片:
with open("./test/Jeux_d'enfants.jpg", 'rb') as f:
    # 设置附件的MIME和文件名，这里是jpg, png类型:
    mime = MIMEBase('image', 'jpg', filename="Jeux_d'enfants.jpg")
    # 加上必要的头信息:
    mime.add_header('Content-Disposition', 'attachment', filename="Jeux_d'enfants.jpg")
    mime.add_header('Content-ID', '<0>')
    mime.add_header('X-Attachment-Id', '0')
    # 把附件的内容读进来:
    mime.set_payload(f.read())
    # 用Base64编码:
    encoders.encode_base64(mime)
    # 添加到MIMEMultipart:
    msg.attach(mime)

发送图片
---
如果要把一个图片嵌入到邮件正文中怎么做？直接在HTML邮件中链接图片地址行不行？答案是，大部分邮件服务商都会自动屏蔽带有外链的图片，因为不知道这些链接是否指向恶意网站。

要把图片嵌入到邮件正文中，我们只需按照发送附件的方式，先把邮件作为附件添加进去，然后，在HTML中通过引用 **src="cid:0"** 就可以把附件作为图片嵌入了。如果有多个图片，给它们依次编号，然后引用不同的 **cid:x** 即可。

把上面代码加入MIMEMultipart的MIMEText从plain改为html，然后在适当的位置引用图片：

In [ ]:
msg.attach(MIMEText('<html><body><h1>Hello</h1>' +
    '<p><img src="cid:0"></p>' +
    '</body></html>', 'html', 'utf-8'))

同时支持HTML和Plain格式
---
如果我们发送HTML邮件，收件人通过浏览器或者Outlook之类的软件是可以正常浏览邮件内容的，但是，如果收件人使用的设备太古老，查看不了HTML邮件怎么办？

办法是在发送HTML的同时再附加一个纯文本，如果收件人无法查看HTML格式的邮件，就可以**自动降级**查看纯文本邮件。

利用**MIMEMultipart**就可以组合一个HTML和Plain，要注意指定**subtype是alternative**。

In [ ]:
msg = MIMEMultipart('alternative')
msg['From'] = ...
msg['To'] = ...
msg['Subject'] = ...

msg.attach(MIMEText('hello', 'plain', 'utf-8'))
msg.attach(MIMEText('<html><body><h1>Hello</h1></body></html>', 'html', 'utf-8'))
# 正常发送msg对象...

加密SMTP
---
使用标准的**25端口**连接SMTP服务器时，使用的是**明文传输**，发送邮件的整个过程可能会被**窃听**。要更安全地发送邮件，可以**加密SMTP会话**，实际上就是**先创建SSL安全连接**，然后再使用SMTP协议发送邮件。

某些邮件服务商，例如Gmail，提供的SMTP服务必须要加密传输。我们来看看如何通过Gmail提供的安全SMTP发送邮件。

Gmail的SMTP端口是587，因此，修改代码如下：

In [ ]:
smtp_server = 'smtp.gmail.com'
smtp_port = 587
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()    # 创建安全连接
# 剩下的代码和前面的一模一样:
server.set_debuglevel(1)
...

只需要在创建SMTP对象后，立刻调用**starttls()**方法，就创建了安全连接。

小结
---
使用Python的smtplib发送邮件十分简单，只要掌握了各种邮件类型的构造方法，正确设置好邮件头，就可以顺利发出。

构造一个邮件对象就是一个Messag对象，如果构造一个MIMEText对象，就表示一个文本邮件对象，如果构造一个**MIMEImage**对象，就表示一个作为附件的图片，要把多个对象组合起来，就用MIMEMultipart对象，而**MIMEBase可以表示任何对象**。它们的继承关系如下：

Message  
&ensp;&ensp;&ensp;&ensp;MIMEBase  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;MIMEMultipart  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;MIMENonMultipart  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;MIMEMessage  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;MIMEText  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;MIMEImage  

这种嵌套关系就可以构造出任意复杂的邮件。

参考文献：https://my.oschina.net/jhao104/blog/613774